In [ ]:
import os
import numpy as np
import discretisedfield as df
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib import cm
from pathlib import Path
from matplotlib.animation import FuncAnimation, PillowWriter

## Instructions

1) Run the cell above to import all of the necessary packages. If you get an error, doublecheck that these are all installed.

2) In the cell below, edit the path to the folder containing your ovf files. Remember that './' refers to the folder this notebook is in. So that would mean that './folder1/folder2' tells python: starting from here, look in folder1, then folder2. You can also use '../' to search in the folder holding this one. Alternatively you can just copy the entire path to the ovf containing folder.

3) Run all of the cells from the very top until you reach the Image Maker section. These cells contain functions that break the whole process into easier to handle pieces.

4) Find the section you want to use (Image/gif maker, single/multi component), edit options found in the cell as necessary and run.

In [ ]:
# Gathers .ovf files and places them into a list
pname = Path('./initialize/Holes/holes_ns_512') # path to the folder containing ".ovf" files
fnames = sorted(pname.glob('*.ovf')) # gets and sorts all the ".ovf" files into a list called fnames

In [ ]:
# Here are some global variables for making plots
norm = colors.CenteredNorm() # this ensures the color map is centered around zero
cmap = cm.coolwarm # color map, can be changed without breaking the script
lbl = ['x','y','z'] # labels for magnetization components

## Functions

In [ ]:
# this function takes an ovf file from the fnames list and returns an array of magnetization values
# the layer option chooses the 2D magnetization at the given layer; if layer=None, integrate mag. along z

def ovf_to_array(fname, layer=None):
    m = df.Field.fromfile(str(fname))
    if layer==None:
        m_array = m.project('z').array[:,:,0,:]
    else:
        m_array = m.array[:,:,layer,:]
    return m_array

In [ ]:
# this function plots one component of the 2D magnetization returned by ovf_to_array
# options to choose layer, mag. component (x=0,y=1,z=2), and image cropping
# crops must be entered as lists of bounds: [xmin,xmax] and [ymin,ymax]

def plot_ovf(fname, layer, component = 2, cropx = [None,None], cropy = [None,None]):
    m = ovf_to_array(fname,layer)
    arr = m[cropy[0]:cropy[1],cropx[0]:cropx[1],component]
    pc = ax.pcolormesh(arr, norm=norm, cmap=cmap)
    pc.set_clim(np.min(arr),np.max(arr))
    ax.set_title(f'Magnetization {lbl[component]}-component ({str(fname)[-6:-4]} - layer{layer})')
    
    plt.show()

In [ ]:
# this function plots all three components of the 2D magnetization returned by ovf_to_array side by side
# options to choose layer and image cropping
# crops must be entered as lists of bounds: [xmin,xmax] and [ymin,ymax]

def plot_ovf_all(fname, layer, cropx = [None,None], cropy = [None,None]):
    m = ovf_to_array(fname, layer)
    arr0 = m[cropy[0]:cropy[1],cropx[0]:cropx[1],0]
    arr1 = m[cropy[0]:cropy[1],cropx[0]:cropx[1],1]
    arr2 = m[cropy[0]:cropy[1],cropx[0]:cropx[1],2]
    
    pc0 = ax0.pcolormesh(arr0, norm=norm, cmap=cmap)
    pc1 = ax1.pcolormesh(arr1, norm=norm, cmap=cmap)
    pc2 = ax2.pcolormesh(arr2, norm=norm, cmap=cmap)

    pc0.set_clim(np.min(arr0), np.max(arr0))
    pc1.set_clim(np.min(arr1), np.max(arr1))
    pc2.set_clim(np.min(arr2), np.max(arr2))
    
    ax0.set_title(lbl[0]+'-component')
    ax1.set_title(lbl[1]+'-component')
    ax2.set_title(lbl[2]+'-component')
    fig.suptitle(f'Magnetization ({str(fname)[-7:-4]} - layer{layer})', fontsize=16)
    
    plt.show()

## Image makers

### single component

In [ ]:
# Checks that at least one ovf file has been found
fname = fnames[0]
print(fname)

In [ ]:
i = 0 # choose which ovf file to process
component = 2 # choose which component of the magnetization to plot
layer = None # choose which layer to plot, layer=None will integrate the magnetization through the material.
cropx = [None,None] # crops the image in the x direction, formatting: cropx=[xmin,xmax]
cropy = [None,None] # crops the image in the y direction, formatting: cropy=[ymin,ymax]
#cropx = [246,266]
#cropy = [0,20]

# this section initializes the figure shape, color map, and color bar
m = ovf_to_array(fnames[i],layer)
arr = m[cropy[0]:cropy[1],cropx[0]:cropx[1],component]
fig, ax = plt.subplots()
pc = ax.pcolormesh(arr, norm=norm, cmap=cmap)
fig.colorbar(pc, ax=ax)

# plots the data in the figure
plot_ovf(fnames[i],layer,component,cropx,cropy)

# saves the figure, commented out by default since I just use the image maker to test before using the gif maker
#fig.savefig('test.png')

### three components

In [ ]:
# same as above, but modified for a 3-wide subplot, each of which also needs to be initialized.
i=0
layer=None
cropx = [None,None]
cropy = [None,None]
#cropx = [246,266]
#cropy = [0,20]

m = ovf_to_array(fnames[0],layer)
arr0 = m[cropy[0]:cropy[1],cropx[0]:cropx[1],0]
arr1 = m[cropy[0]:cropy[1],cropx[0]:cropx[1],1]
arr2 = m[cropy[0]:cropy[1],cropx[0]:cropx[1],2]

fig, (ax0,ax1,ax2) = plt.subplots(1,3, figsize=(20,6),constrained_layout=True)
pc0 = ax0.pcolormesh(arr0, norm=norm, cmap=cmap)
pc1 = ax1.pcolormesh(arr1, norm=norm, cmap=cmap)
pc2 = ax2.pcolormesh(arr2, norm=norm, cmap=cmap)
fig.colorbar(pc0, ax=ax0)
fig.colorbar(pc1, ax=ax1)
fig.colorbar(pc2, ax=ax2)

plot_ovf_all(fnames[i], layer, cropx, cropy)

# saves the figure, commented out by default since I just use the image maker to test before using the gif maker
#fig.savefig('test.png')

## Gif-makers

### single component

In [ ]:
# same as above
layer = None
component = 2
cropx = [None,None]
cropy = [None,None]
#cropx = [246,266]
#cropy = [0,20]

# same as above
m = ovf_to_array(fnames[0],layer)
arr = m[cropy[0]:cropy[1],cropx[0]:cropx[1],component]
fig, ax = plt.subplots()
pc = ax.pcolormesh(arr, norm=norm, cmap=cmap)
fig.colorbar(pc, ax=ax)

# This block uses matplotlib FuncAnimation, which writes a frame by calling the function plot_ovf on an entry in the fnames list.
ani = FuncAnimation(fig, plot_ovf, fnames, fargs=[layer,component,cropx,cropy])
writer = PillowWriter(fps=2)
ani.save(f'test_{lbl[component]}.gif', writer=writer)

### three components

In [ ]:
# same as above
layer = None
#cropx = [None,None]
#cropy = [None,None]
cropx = [246,266]
cropy = [0,20]

m = ovf_to_array(fnames[0],layer)
arr0 = m[cropy[0]:cropy[1],cropx[0]:cropx[1],0]
arr1 = m[cropy[0]:cropy[1],cropx[0]:cropx[1],1]
arr2 = m[cropy[0]:cropy[1],cropx[0]:cropx[1],2]

fig, (ax0,ax1,ax2) = plt.subplots(1,3, figsize=(20,6),constrained_layout=True)
pc0 = ax0.pcolormesh(arr0, norm=norm, cmap=cmap)
pc1 = ax1.pcolormesh(arr1, norm=norm, cmap=cmap)
pc2 = ax2.pcolormesh(arr2, norm=norm, cmap=cmap)
fig.colorbar(pc0, ax=ax0)
fig.colorbar(pc1, ax=ax1)
fig.colorbar(pc2, ax=ax2)

ani = FuncAnimation(fig, plot_ovf_all, fnames, fargs = [layer,cropx,cropy])
writer = PillowWriter(fps=2)
ani.save('./gifs/hole gifs/holes_ns_512.gif', writer=writer)